In [7]:
# Importing Libraries including NFLfastpy
import nfl_data_py as nfl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)

In [42]:
#Importing play-by-play data to dataframe
years = [2023]
pbp_df = nfl.import_pbp_data(years, downcast=True, cache=False)
wkly_df = nfl.import_weekly_data(years, downcast=True)
szn_df = nfl.import_seasonal_data(years)
ros_df = nfl.import_seasonal_rosters(years)

2023 done.
Downcasting floats.
Downcasting floats.


In [185]:
pbp_column_list = [column for column in nfl.see_pbp_cols()]
wkly_column_list = [column for column in nfl.see_weekly_cols()]

In [229]:
#array(['GAME_START', 'KICK_OFF', 'RUSH', 'PASS', 'PUNT', 'PENALTY',
#        'XP_KICK', 'TIMEOUT', 'FIELD_GOAL', 'SACK', 'END_QUARTER',
#        'INTERCEPTION', 'UNSPECIFIED', 'END_GAME',
#        'FUMBLE_RECOVERED_BY_OPPONENT', 'PAT2', 'COMMENT']

#cleaning up missing values and play types that are irrelevant

play_types = ['RUSH', 'PASS', 'PUNT', 'PENALTY','SACK','INTERCEPTION', 'FUMBLE_RECOVERED_BY_OPPONENT']
df = pbp_df[pbp_df['play_type_nfl'].isin(play_types)]

In [ ]:
#investigating yards_gain == NaN values
plays = df[df["yards_gained"].isnull()]
plays = df[df["yards_gained"].isnull()]["desc"].tolist()
plays

In [230]:
#remove errenous plays (e.g. 30 plays with NaN as yards gained)
df = df[df["down"].notnull()]
df = df[df["yards_gained"].notnull()]

In [220]:
df.duplicated().sum()

0

In [231]:
#checking play types in current df
df['play_type_nfl'].unique()

array(['RUSH', 'PASS', 'PUNT', 'PENALTY', 'SACK', 'INTERCEPTION',
       'FUMBLE_RECOVERED_BY_OPPONENT'], dtype=object)

In [232]:
df["play_type"].unique()

array(['run', 'pass', 'punt', 'no_play', 'qb_kneel', 'qb_spike'],
      dtype=object)

In [248]:
#data types of features
df.dtypes

play_id                                 float32
game_id                                  object
old_game_id                              object
home_team                                object
away_team                                object
season_type                              object
week                                      int32
posteam                                  object
posteam_type                             object
defteam                                  object
side_of_field                            object
yardline_100                              int64
game_date                                object
quarter_seconds_remaining               float32
half_seconds_remaining                  float32
game_seconds_remaining                  float32
game_half                                object
quarter_end                             float32
drive                                   float32
sp                                      float32
qtr                                     

In [245]:
#casting downs, yard line, yards to go, etc. to integers from floats
df = df.astype({"down": 'int',
                'first_down': 'int',
                "yardline_100": 'int',
                "yards_gained": 'int',
                "ydstogo" : 'int',
                "third_down_converted": 'int'
                })

In [ ]:
first_down_df = df[df['down'] == 1]
first_down_df = first_down_df[['game_id', 'quarter_seconds_remaining', 'week', 'posteam_type', 'posteam', 'defteam', 'down', 'first_down', 'yardline_100', 'ydstogo', 'yards_gained', 'play_type', 'play_type_nfl', 'fantasy_player_name','desc']]
first_down_df[first_down_df["posteam"] == "CIN"]

In [258]:
#Filtering down to 3rd down plays and relevant features
third_down_df = df[df['down'] == 3]
third_down_df = third_down_df[['game_id', 'week', 'posteam_type', 'posteam', 'defteam', 'down', 'first_down', 'yardline_100', 'ydstogo', 'yards_gained', 'play_type', 'play_type_nfl', 'fantasy_player_name','desc']]

In [280]:
third_down_df[third_down_df["defteam"] == "CIN"]["yards_gained"].sum()/third_down_df[third_down_df["defteam"] == "CIN"]["yards_gained"].count()

5.279329608938547

In [283]:
weeks = [12,13,14]
rec_third_df = third_down_df[third_down_df["week"].isin(weeks)]

In [285]:
third_down_df[third_down_df["defteam"] == "CIN"]["yards_gained"].sum()/third_down_df[third_down_df["defteam"] == "CIN"]["yards_gained"].count()

5.279329608938547

In [286]:
rec_third_df[rec_third_df["defteam"] == "CIN"]["yards_gained"]

30835    10
30843     3
30846    29
30851     1
30863    39
30866     0
30878     0
30889     0
30905     9
30924     5
30927     9
30932     8
30948    12
30952    43
30955     0
30969     7
30982    -1
32033     2
32047     0
32065     0
32066     1
32093     0
32100     0
32110     0
32111    16
32114     3
32120    14
32126     0
32138     0
32139     0
32140     0
32162    -7
32177    14
32183    -2
32195    10
32199     0
32200    -4
34859     0
34869     0
34887     4
34909     9
34913    12
34916     6
34919     2
34923     0
34956     0
34974     0
35009     2
Name: yards_gained, dtype: int64